<a href="https://colab.research.google.com/github/crackerbad/colab-tools/blob/main/Colab_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Login

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# Timeout

In [ ]:
#@markdown <br><center><img src='https://miro.medium.com/max/2036/1*GW4cqNv-SIQ9BGGhssL1bQ.png' height="100" alt="netdata"/></center>
#@title <center><b>Time Out Preventer (Advanced) </b></center></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

# Requerimentos

In [2]:
#@markdown <br><center><img src='https://i.imgur.com/VvjOf7S.png' height="150" alt="CR-logo"/></center>
#@markdown <center><h3>Instalar Ferramentas</h3></center><br>
!curl https://raw.githubusercontent.com/crackerbad/colab-tools/main/colab-tools.sh | bash

from IPython.display import clear_output
clear_output()

<br><br><br><br><br><br><br>

# <center><img src='https://logodownload.org/wp-content/uploads/2020/02/crunchyroll-logo.png' height="50" alt="CR-logo"/>
<center><h3>Downloader</h3></center><br>

LISTA-DUB.TXT

In [ ]:
%%writefile lista-dub.txt
https://www.crunchyroll.com/pt-br/requiem-of-the-rose-king/episode-12-richard-is-dead-died-alone-in-the-woods-that-day-843391


LISTA-LEG.TXT

In [ ]:
%%writefile lista-leg.txt
https://www.crunchyroll.com/pt-br/requiem-of-the-rose-king/episode-12-richard-is-dead-died-alone-in-the-woods-that-day-842369

In [ ]:
#@title BAIXAR LEGENDADO
!yt-dlp  --progress -q --no-warnings --cookies cookies.txt -o "%(series)s - %(episode_number)02d [WEB-DL 1080p] [CR].%(ext)s"  --write-sub --sub-lang ptBR --sub-format ass -f b -a "lista-leg.txt"
!for f in *.mp4 ; do mkvmerge -o "Download/${f%.mp4}.mkv" --track-name "1:JAP AAC 2.0" --language 1:ja "$f" --track-name "0:Crunchyroll FULL" --language "0:por" "${f%.mp4}.ptBR.ass" ; rm "$f" ; rm "${f%.mp4}.ptBR.ass" ;done

In [ ]:
#@title BAIXAR DUAL-AUDIO
#baixando audio dublado
!yt-dlp -f bestaudio -o "temp/%(series)s - %(episode_number)02d [CR] [WEB-DL "$quality"p].%(ext)s" --cookies "cookies.txt" -a "lista-dub.txt" --write-sub --sub-lang pt-BR --sub-format ass --fragment-retries 20 -N 16 --extractor-args crunchyrollbeta:format=adaptive_dash
#arrumando nomes
!for file in ./*; do mv "$file" "${file/ (*)/}"; done
#extraindo audios
!for f in *.mp4 ; do mkvmerge -o "${f%.m4a}.mka" --track-name "0:PT-BR AAC 2.0" --language 0:pt --no-video "$f" --track-name "0:Crunchyroll Forced" --language "0:pt" --default-track "0:yes" "${f%.m4a}.pt-BR.ass"  ; rm "$f" ; rm "${f%.m4a}.ptBR.ass" ;done
#removendo partes do nome
!rename 's= \(Dubs\)==' temp/*
!rename 's= \(Dublado\)==' temp/*

#baixando video com audio japones e legendas
!yt-dlp -f best[height=$quality] -o "temp/%(series)s - %(episode_number)02d [CR] [WEB-DL "$quality"p].%(ext)s" --cookies "cookies.txt" -a "lista-leg.txt" --write-sub --sub-lang pt-BR --sub-format ass --fragment-retries 20 --external-downloader aria2c --external-downloader-args "aria2c:-x4"
#juntando legenda+video
%cd temp
!for f in *.mp4 ; do mkvmerge -o "${f%.mp4}.mkv" --track-name "1:JAP AAC 2.0" --language 1:ja "$f" --track-name "0:Crunchyroll FULL" --language "0:pt" "${f%.mp4}.pt-BR.ass" ; rm "$f" ; rm "${f%.mp4}.ptBR.ass" ;done
#finalizando juntando tudo
!for f in *.mka ; do mkvmerge -o "../Downloads/${file%.mka} [Dual-Audio].mkv" "$file" "${file%.mka}.mkv" --track-order 1:0,0:0,1:1,0:1,1:2 ; rm "$file" ; rm "${file%.mka}.mkv" ;done
%cd ..
!rmdir temp/

<br><br><br><br><br>

# <center><img src='https://www.seekpng.com/png/full/774-7747311_ummy-youtube-downloader-for-mac-ummy-video-downloader.png' height="200" alt="Multi-logo"/>
<center><h3>Downloader</h3></center><br>

In [ ]:
# User Input
Source = "https://kumo.anitsu.moe/nextcloud/s/KfmLyiSwQFabsBf" #@param {type:"string"} 
Mode = "Anitsu Folder" #@param ["Google Drive File", "Google Drive Folder","OneDrive Sharepoint","Anitsu ZIP","Anitsu Folder","Google Drive Index", "Torrent (Magnet ou .Torrent)"]
Destination = "/content/" #@param {type:"string"} 
# Task Completed Message
output = "Task Completed Successfully!"
# Work
if(Mode=="Google Drive File"):
  %cd {Destination}
  !gdown {Source}
  print(output)
  %cd /content
if(Mode=="Google Drive Folder"):
  %cd {Destination}
  !gdown {Source} --folder
  print(output)
  %cd /content
if(Mode=="OneDrive Sharepoint"):
  %cd {Destination}
  !sh /content/odrive/odrive-downloader.sh {Source}
  %cd /content
if(Mode=="Anitsu ZIP"):
  %cd /content
  !wget {Source} -O Download.zip
  !unzip Download.zip
  !rm Download.zip
if(Mode=="Anitsu Folder"):
  %cd {Destination}
  !chmod u+r+x /content/nextcloud/nextcloud_share_url_downloader.sh
  !bash /content/nextcloud/nextcloud_share_url_downloader.sh {Source} pasta pasta
  %cd /content
if(Mode=="Google Drive Index"):
  %cd {Destination}
  !python3 /content/nextcloud/index-downloader.py -u {Source}
  %cd /content
if(Mode=="Torrent (Magnet ou .Torrent)"):
  %cd {Destination}
  !aria2c --seed-time=0 --auto-save-interval=0 {Source} -d "{Destination}" --disable-ipv6
  !rm *.torrent
  %cd /content

In [ ]:
!df -H